In [2]:
import numpy as np
import sys
import pickle
import matplotlib.pyplot as plt
import sklearn

In [3]:
import pandas as pd

In [4]:

tags = pd.read_csv('dataset/tag.csv')
ratings = pd.read_csv('dataset/rating.csv')
links = pd.read_csv('dataset/link.csv')

In [5]:
movies = pd.read_csv('dataset/movie.csv',encoding='ISO-8859-1')

In [6]:
movies.head()

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN,NaN
1,2,jumanji,Adventure|Children|Fantasy,NaN,NaN,NaN,NaN
2,3,grumpier old men,Comedy|Romance,NaN,NaN,NaN,NaN
3,4,waiting to exhale,Comedy|Drama|Romance,NaN,NaN,NaN,NaN
4,5,father of the bride part ii,Comedy,NaN,NaN,NaN,NaN


In [7]:
tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [9]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [10]:
movies.shape

(27278, 7)

In [11]:
links.shape

(27278, 3)

In [12]:
ratings.shape

(20000263, 4)

In [13]:
movies.head()

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN,NaN,NaN
1,2,jumanji,Adventure|Children|Fantasy,NaN,NaN,NaN,NaN
2,3,grumpier old men,Comedy|Romance,NaN,NaN,NaN,NaN
3,4,waiting to exhale,Comedy|Drama|Romance,NaN,NaN,NaN,NaN
4,5,father of the bride part ii,Comedy,NaN,NaN,NaN,NaN


In [14]:
movies['genres']=movies['genres'].str.replace('|',' ')

C:\Users\Admin\AppData\Local\Temp\ipykernel_14488\2665461764.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies['genres']=movies['genres'].str.replace('|',' ')


In [15]:
movies.head()

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN
1,2,jumanji,Adventure Children Fantasy,NaN,NaN,NaN,NaN
2,3,grumpier old men,Comedy Romance,NaN,NaN,NaN,NaN
3,4,waiting to exhale,Comedy Drama Romance,NaN,NaN,NaN,NaN
4,5,father of the bride part ii,Comedy,NaN,NaN,NaN,NaN


In [16]:
len(movies.movieId.unique())

27278

In [17]:
len(ratings.movieId.unique())

26744

In [18]:
#removing user who has rated less than 350 movies
ratings_F = ratings.groupby('userId').filter(lambda x: len(x)>=350)

#remaning movies
movie_list_rating = ratings_F.movieId.unique().tolist()

In [19]:
#we have kept 96.4%movies
len(ratings_F.movieId.unique())/len(movies.movieId.unique())*100

96.37070166434489

In [20]:

len(ratings_F.userId.unique())/len(ratings.userId.unique())*100

9.396864823492884

In [21]:
movies = movies[movies.movieId.isin(movie_list_rating)]

In [22]:
movies.shape

(26288, 7)

In [23]:
len(ratings_F.movieId.unique())

26288

In [24]:
movies.head()

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN
1,2,jumanji,Adventure Children Fantasy,NaN,NaN,NaN,NaN
2,3,grumpier old men,Comedy Romance,NaN,NaN,NaN,NaN
3,4,waiting to exhale,Comedy Drama Romance,NaN,NaN,NaN,NaN
4,5,father of the bride part ii,Comedy,NaN,NaN,NaN,NaN


In [25]:
#mapping movie to id
Mapping_file = dict(zip(movies.title.tolist(),movies.movieId.tolist()))

In [26]:
tags.drop(['timestamp'],1,inplace = True)
ratings_F.drop(['timestamp'],1,inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14488\2416006150.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  tags.drop(['timestamp'],1,inplace = True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_14488\2416006150.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings_F.drop(['timestamp'],1,inplace = True)


In [27]:
#merge movies and tag
mixed = pd.merge(movies,tags,on = 'movieId', how = 'left')
mixed.head(10)

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,userId,tag
0,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,1644.0,Watched
1,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,1741.0,computer animation
2,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,1741.0,Disney animated feature
3,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,1741.0,Pixar animation
4,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,1741.0,TÃ©a Leoni does not star in this movie
5,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,2299.0,Pixar
6,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,3596.0,animation
7,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,3596.0,family
8,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,3596.0,Tom Hanks
9,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,6233.0,Pixar


In [28]:
#create metadata from genre and tags
mixed.fillna("",inplace = True)
mixed = pd.DataFrame(mixed.groupby("movieId")['tag'].apply(lambda x: "%s" % ' '.join(x)))
Final = pd.merge(movies,mixed ,on= "movieId", how = "left")
Final['metadata'] = Final[['tag','genres']].apply(lambda x: ' '.join(x),axis = 1)
Final[['movieId','title','metadata']].head()

,movieId,title,metadata
0,1,toy story,Watched computer animation Disney animated fea...
1,2,jumanji,time travel adapted from:book board game child...
2,3,grumpier old men,old people that is actually funny sequel fever...
3,4,waiting to exhale,chick flick revenge characters chick flick cha...
4,5,father of the bride part ii,Diane Keaton family sequel Steve Martin weddin...


In [29]:
Final.shape

(26288, 9)

In [30]:
Final_1 = pd.merge(Final,links, on='movieId',how = 'left')
Final_1.head()

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,tag,metadata,imdbId,tmdbId
0,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,Watched computer animation Disney animated fea...,Watched computer animation Disney animated fea...,114709,862.0
1,2,jumanji,Adventure Children Fantasy,NaN,NaN,NaN,NaN,time travel adapted from:book board game child...,time travel adapted from:book board game child...,113497,8844.0
2,3,grumpier old men,Comedy Romance,NaN,NaN,NaN,NaN,old people that is actually funny sequel fever...,old people that is actually funny sequel fever...,113228,15602.0
3,4,waiting to exhale,Comedy Drama Romance,NaN,NaN,NaN,NaN,chick flick revenge characters chick flick cha...,chick flick revenge characters chick flick cha...,114885,31357.0
4,5,father of the bride part ii,Comedy,NaN,NaN,NaN,NaN,Diane Keaton family sequel Steve Martin weddin...,Diane Keaton family sequel Steve Martin weddin...,113041,11862.0


In [31]:
Final_1.drop(['imdbId'],1,inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14488\3920020846.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Final_1.drop(['imdbId'],1,inplace = True)


In [32]:
Final_1.head()

,movieId,title,genres,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,tag,metadata,tmdbId
0,1,toy story,Adventure Animation Children Comedy Fantasy,NaN,NaN,NaN,NaN,Watched computer animation Disney animated fea...,Watched computer animation Disney animated fea...,862.0
1,2,jumanji,Adventure Children Fantasy,NaN,NaN,NaN,NaN,time travel adapted from:book board game child...,time travel adapted from:book board game child...,8844.0
2,3,grumpier old men,Comedy Romance,NaN,NaN,NaN,NaN,old people that is actually funny sequel fever...,old people that is actually funny sequel fever...,15602.0
3,4,waiting to exhale,Comedy Drama Romance,NaN,NaN,NaN,NaN,chick flick revenge characters chick flick cha...,chick flick revenge characters chick flick cha...,31357.0
4,5,father of the bride part ii,Comedy,NaN,NaN,NaN,NaN,Diane Keaton family sequel Steve Martin weddin...,Diane Keaton family sequel Steve Martin weddin...,11862.0


In [33]:
Final_1.drop(['genres','tag'],1,inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14488\4273958896.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Final_1.drop(['genres','tag'],1,inplace = True)


In [34]:
Final_1.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],1,inplace = True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_14488\3543812901.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  Final_1.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],1,inplace = True)


In [35]:
Final_1.head()
#content

,movieId,title,metadata,tmdbId
0,1,toy story,Watched computer animation Disney animated fea...,862.0
1,2,jumanji,time travel adapted from:book board game child...,8844.0
2,3,grumpier old men,old people that is actually funny sequel fever...,15602.0
3,4,waiting to exhale,chick flick revenge characters chick flick cha...,31357.0
4,5,father of the bride part ii,Diane Keaton family sequel Steve Martin weddin...,11862.0


In [36]:
Final.loc[1,'metadata']

'time travel adapted from:book board game childhood recaptured game herds of CGI animals scary see also:Zathura time time travel board game fantasy Robin Williams scary time travel Robin Williams Joe Johnston Robin Williams kid flick jungle Robin Williams board game Robin Williams animals Lebbat Robin Williams time travel adventure Robin Williams Children Fantasy Robin Williams Dynamic CGI Action Kirsten Dunst Robin Williams Robin Williams fantasy kid flick animals animals fantasy For children fantasy adapted from:book childish Children kid flick time travel Joe Johnston fantasy Robin Williams time travel animals board game children fantasy Kirsten Dunst Robin Williams time travel board game time travel Children kid flick Filmed in BC fantasy Robin Williams animals bad cgi based on a book board game Chris Van Allsburg Robin Williams Robin Williams game magic board game monkey adapted from:book animals bad cgi based on a book board game childhood recaptured Children Chris Van Allsburg f

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(Final['metadata'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),index = Final.index.tolist())
print(tfidf_df.shape)

(26288, 23627)


In [38]:
tfidf_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,23617,23618,23619,23620,23621,23622,23623,23624,23625,23626
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.440618,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 200)
latent_matrix = svd.fit_transform(tfidf_df)
#plot graph
explained = svd.explained_variance_ratio_.cumsum()
plt.plot(explained, '.-',ms = 16,color='red')
plt.xlabel('singular value components',fontsize=12)
plt.ylabel('cumulative percent of variance',fontsize=12)
plt.show()

In [ ]:
#number of latent dimension to keep
n = 200
latent_matrix_1_df = pd.DataFrame(latent_matrix[:,0:n],index = Final.title.tolist())

In [ ]:
latent_matrix.shape

In [ ]:
latent_matrix[0]

In [ ]:
#creating collabrative latent matrix from user rating
ratings_F.head()

In [ ]:
rating_f1 = pd.merge(movies["movieId"],ratings_F, on ="movieId", how="right")

In [ ]:
rating_f1.head()

In [ ]:
rating_f2  = rating_f1.pivot(index = 'movieId',columns = 'userId',values='rating').fillna(0)

In [ ]:
rating_f2.head()

In [ ]:
rating_f2.shape

In [ ]:
len(ratings_F.movieId.unique())

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 200)
latent_matrix_2 = svd.fit_transform(rating_f2)
latent_matrix_2_df = pd.DataFrame(
                            latent_matrix_2,
                            index=Final.title.tolist())

In [ ]:
latent_matrix_2_df.shape

In [ ]:
explained = svd.explained_variance_ratio_.cumsum()
plt.plot(explained,'.-',ms = 16,color = 'red')
plt.xlabel('singular value components', fontsize=12)
plt.ylabel('cumulative percent of variance', fontsize=12)
plt.show()

In [ ]:
latent_matrix_1_df.head()

In [ ]:
latent_matrix_2_df.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#take latent vector for selectred movie from both and collebrative matrices
a_1 = np.array(latent_matrix_1_df.loc['spider-man ']).reshape(1,-1)
a_2 = np.array(latent_matrix_2_df.loc['spider-man ']).reshape(1,-1)

#calculate similarty of movies with other in list
score_1 = cosine_similarity(latent_matrix_1_df, a_1).reshape(-1)
score_2 = cosine_similarity(latent_matrix_2_df, a_2).reshape(-1)

#an average measure of both content and collabrative
hybrid = ((score_1+score_2)/2.0)

#from data frame of similar movies
dictDf = {'content':score_1,'collabrative':score_2,'hybrid':hybrid}
similar = pd.DataFrame(dictDf, index = latent_matrix_1_df.index)

#sort it on basis of content hybrid or collabrative
similar.sort_values('hybrid',ascending=False,inplace=True)

recommended_movie = similar[1:6].index.tolist()
movie_list_id = []
for m in recommended_movie:
    movie_list_id.append(int(Final_1[Final_1["title"] == m]["tmdbId"]))
#     print(Final_1[Final_1["title"] == m])

print(recommended_movie)
print(movie_list_id)
similar.head(6)

In [ ]:
import pickle

In [ ]:
pickle.dump(Final_1,open('model/Final.pkl','wb'))

In [ ]:
pickle.dump(latent_matrix_1_df,open('model/latent_matrix_1.pkl','wb'))

In [ ]:
pickle.dump(latent_matrix_2_df,open('model/latent_matrix_2.pkl','wb'))